# Neo4j + MITRE ATT&CK: Hands-On Tutorial

**Learning Goals**
- Understand the ATT&CK knowledge graph schema: nodes, relationships, and STIX types
- Run Level-1 lookups, Level-2 multistep analytics, and Level-3 graph-algorithm questions
- Interpret results for threat-defense decision-making (tactics, mitigations, high-risk software)

**How to Use This Notebook**
- Each section starts with a short **approach** and ends with **validation** tips
- Use the provided helper functions (`query`, `list_*`, `kg_*`, etc.) to keep Cypher tidy


In [1]:
import sys
!{sys.executable} -m pip install neo4j



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip


# Connect to Neo4j from Python — what this cell does (and why)

This notebook uses the **official Neo4j Python Driver** to connect to your database over the **Bolt** protocol, run parameterized Cypher, and return results as pandas DataFrames. The Python driver exposes the same core concepts as other Neo4j drivers (Driver → Session → Transaction) and is the recommended way to build apps or notebooks against Neo4j.

---

## Connection basics

- **URI & routing.** `neo4j://` enables the driver’s **routing** behavior (works for single servers and clusters). If you explicitly want a direct one-to-one connection, use `bolt://` (or `bolt+s` / `bolt+ssc` for TLS variants). Keep `neo4j://` unless you have a reason to force direct connections. :contentReference[oaicite:1]{index=1}  
- **Auth.** Supply `("user","password")`; the driver handles connection pooling for you. 


## What each function/variable below does

- **CONFIG (URI, USER, PASS, DEFAULTS).** Central place to adjust connection info and default values for common parameters (group, technique, etc.).

- **`driver = GraphDatabase.driver(...)`**  
  Creates a **Driver** with a connection pool. Keep one per process and reuse it; close it when done.

- **`close()`**  
  Closes the driver/pool cleanly (good practice in notebooks when you’re finished).

- **`df_from_result(result)`**  
  Converts the driver’s `Result` (an iterator of records) into a list of dicts and then a pandas **DataFrame**—handy for tabular display.

- **`query(q: str, **params)`**  
  Opens a short-lived **Session**, runs Cypher with **parameters**, and returns a DataFrame. This uses **auto-commit** under the hood (`session.run`), which is ideal for interactive exploration. For production code that needs automatic retries, prefer `session.execute_read/write`.

---

## Usage tips

- Reuse the **Driver**, not Sessions. Create short Sessions inside helper functions (as shown) to avoid keeping transactions open.
- Always pass **parameters** (`$param`) instead of formatting values into the Cypher string. It’s safer and lets the planner cache execution plans.
- If you later add long-running writes or multi-step logic, switch to **managed transactions** (`execute_write/execute_read`) to get automatic retry semantics. 
- The same code works with **Neo4j Desktop**, **Docker/Podman**, **Aura**, and clusters. Just change `NEO4J_URI`, `NEO4J_USER`, `NEO4J_PASS`.

---

## Quick reference

- **Python driver manual (connect, sessions, transactions):**  
  https://neo4j.com/docs/python-manual/current/  
  *Official guide to creating a Driver, opening Sessions, and running queries.* 

- **Managed transactions (`execute_read` / `execute_write`):**  
  https://neo4j.com/docs/python-manual/current/performance/  
  *Shows transaction functions that auto-retry on transient failures.* 


In [7]:
from neo4j import GraphDatabase
import pandas as pd
from typing import Dict, Any, List

# ---------- CONFIG ----------
NEO4J_URI  = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASS = "YourStrongPass123!"

DEFAULTS = {
    "group": "APT29",
    "tech": "Credential Dumping",
    "tactic": "defense-evasion",
    "g1": "APT29",
    "g2": "FIN7",
    "software": "Mimikatz",
    "recent_days": 180,
}

# ---------- UTILS ----------

driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j","YourStrongPass123!"))

def close():
    driver.close()

def df_from_result(result) -> pd.DataFrame:
    rows = [dict(r) for r in result]
    return pd.DataFrame(rows)


def query(q: str, **params) -> pd.DataFrame:
    with driver.session() as s:
        res = s.run(q, **params)
        return df_from_result(res)

def _print_section(title: str, df: pd.DataFrame):
    print(f"\\n=== {title} ===")
    if df is None:
        print("(no results)")
        return
    if df.empty:
        print("(empty)")
    else:
        # limit very wide cells for tebrminal readability
        with pd.option_context("display.max_colwidth", 80, "display.width", 140):
            print(df.to_string(index=False))


# MITRE ATT&CK (STIX 2.1) — Knowledge Base → Neo4j Knowledge Graph

This section documents **what the dataset is**, **how we turn it into a property graph**, and **why it matters** for security analytics and attack-surface reasoning.

---

## 1) What is this dataset?

**MITRE ATT&CK** is an open, curated knowledge base of **adversary behavior**. It captures:
- **Who**: adversary *groups* (intrusion-sets) and *campaigns*
- **How**: *techniques* and *sub-techniques* adversaries use
- **With what**: *tools* and *malware*
- **How to defend**: *mitigations* (courses of action), *data sources* and *data components* for detection

MITRE publishes ATT&CK as **STIX 2.1 JSON** collections. The official repo you cloned contains three top-level bundles (https://github.com/mitre-attack/attack-stix-data):

- enterprise-attack/enterprise-attack.json
- mobile-attack/mobile-attack.json
- ics-attack/ics-attack.json



Each bundle is a **graph** of typed objects + explicit relationship objects.

---

## 2) What’s inside (interesting details)

### Object types you’ll see most often
- `attack-pattern` → **techniques** and **sub-techniques**
- `x-mitre-tactic` → **tactics** (the columns in the ATT&CK matrix)
- `intrusion-set` → **groups** (APT, FIN*, etc.)
- `tool`, `malware` → **software** used by groups
- `course-of-action` → **mitigations**
- `x-mitre-data-source`, `x-mitre-data-component` → **detection** concepts
- plus `campaign`, `relationship`, and other STIX objects

### Key relationship semantics (from STIX `relationship` objects)
- `uses` — group→technique, group→software, software→technique
- `mitigates` — course-of-action→technique
- `detects` — data-source/data-component→technique
- `subtechnique-of` — technique hierarchy
- many others (e.g., `compromises`, `targets`, etc., depending on object types)

### Rich metadata you can query
- `x_mitre_platforms` (Windows, Linux, macOS, Azure, AWS, GCP, SaaS, Network…)
- `kill_chain_phases` (e.g., `defense-evasion`, `credential-access`), which link techniques to tactics
- `revoked` / `x_mitre_deprecated` flags (for currency)
- `created` / `modified` timestamps (for recency)
- Domains (enterprise, mobile, ics)

> **Why this is nice for graphs:** ATT&CK is already a **linked** knowledge base with typed nodes and edges. That makes it perfect for a property-graph like Neo4j, where you can ask “Who uses what, which tactic is affected, and how do we mitigate/detect it?” with a few hops.

---

## 3) Our Neo4j graph model (Labeled Property Graph)

We import the STIX bundles into a streamlined **property graph**:

**Nodes**
- Label: `:Attack`
- Properties (subset):  
  `id, stix_type, name, description, created, modified, revoked, deprecated, platforms[], domains[], kc_phases[], shortname`

**Relationships**
- Type: `:ATTACK_REL` (1:1 with STIX *relationship* objects)  
  Properties: `id, rel_type, description`  
  Examples: `rel_type = 'uses' | 'mitigates' | 'detects' | 'subtechnique-of' | …`

**Helper edges**
- `(:Attack {stix_type:'attack-pattern'})-[:IN_TACTIC]->(:Attack {stix_type:'x-mitre-tactic'})`  
  Derived from `kill_chain_phases` to make tactic queries fast and readable.

This keeps the graph faithful to STIX while giving you intuitive patterns to query.

---

## 4) How we create the knowledge graph (repeatable steps)

> You can run this from the Neo4j Browser or the Python driver. It assumes your JSON file is in Neo4j’s `/import` (e.g., `file:///enterprise-attack/enterprise-attack.json`) and APOC is enabled.

**(a) Constraint (idempotent)**
```cypher
CREATE CONSTRAINT attack_id IF NOT EXISTS
FOR (n:Attack) REQUIRE n.id IS UNIQUE;


## Part A - Orientation

#### How big is the graph?

In [9]:
def A1_counts():
    q1 = "MATCH (n:Attack) RETURN count(n) AS nodes"
    q2 = "MATCH ()-[r:ATTACK_REL]->() RETURN count(r) AS relationships"
    return query(q1), query(q2)

In [11]:
nodes_df, rels_df = A1_counts()

_print_section("A1 counts — nodes", nodes_df)
_print_section("A1 counts — relationships", rels_df)

\n=== A1 counts — nodes ===
 nodes
  2242
\n=== A1 counts — relationships ===
 relationships
         20411


## What is a Knowledge Graph?

A **knowledge graph** = a graph whose nodes/edges are **typed** and **semantically meaningful** (based on a domain schema/ontology), curated so you can ask **who/what/why/how** questions.

The **MITRE ATT&CK** graph fits that perfectly: entities like **groups, techniques, tools, mitigations, tactics** and relationships like **uses, mitigates, detects, subtechnique-of**. It encodes **domain knowledge**, not just raw events.

---

## What are h, r, t?

They’re the classic triple components:

- **h (head)** = the starting entity (**subject**)
- **r (relation)** = the relationship/verb (**predicate**)
- **t (tail)** = the ending entity (**object**)

**In ATT&CK terms:**
- `(h,r,t) = ("APT29", "uses", "Bypass User Account Control")`
- `(h,r,t) = ("Bypass User Account Control", "IN_TACTIC", "defense-evasion")`
- `(h,r,t) = ("Mimikatz", "uses", "Credential Dumping")`
- `(h,r,t) = ("Application Control", "mitigates", "Bypass User Account Control")`

---

## How that maps to Neo4j (property graph)

- **h → node**
  ```cypher
  (:Attack {name:"APT29", stix_type:"intrusion-set"})
- **r → relationship**
  ```cypher
  ([:ATTACK_REL {rel_type:"uses"}]    // or
    [:IN_TACTIC])
- **t → node**
  ```cypher
    (:Attack {name:"Bypass User Account Control", stix_type:"attack-pattern"})



## Part 0 — ATT&CK Knowledge Graph: What’s inside

**Node label**  
- `:Attack` — all ATT&CK objects live here; disambiguated by `stix_type`

**STIX types you will use**
- `attack-pattern` (techniques & sub-techniques)
- `x-mitre-tactic` (tactics / matrix columns)
- `intrusion-set` (adversary groups)
- `tool`, `malware` (software)
- `course-of-action` (mitigations)

**Key relationships**
- `:ATTACK_REL {rel_type:'uses'}` — group→technique, group→software, software→technique
- `:ATTACK_REL {rel_type:'mitigates'}` — mitigation→technique
- `:ATTACK_REL {rel_type:'detects'}` — detection source/component → technique
- `:ATTACK_REL {rel_type:'subtechnique-of'}` — technique hierarchy
- `:IN_TACTIC` — technique → tactic (derived for convenience)

> **Tip**: “Software” = both `tool` **and** `malware`.


# Level 1 — Easy (schema & lookups)

### Q1. Labels Overview
**Question**: List all labels and their counts, sorted by volume.  
**Approach**: Call `kg_labels()` and `kg_node_counts_by_label()` and show top entries.  
**Why**: Sanity-check what’s in the DB and what’s relevant to ATT&CK work.  
**Validate**: You should see `Attack` with ~2k nodes based on dataset.

**Run**:
- `kg_labels()`
- `kg_node_counts_by_label()`


In [ ]:
# ============================================
# Schema inventory (no APOC, no SELECT hacks)
# ============================================

def kg_labels():
    return query("CALL db.labels() YIELD label RETURN label ORDER BY label")

def kg_relationship_types():
    return query("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType ORDER BY relationshipType")

def kg_node_counts_by_label():
    """
    Count nodes per label; multi-labeled nodes are counted once per label.
    """
    q = """
    MATCH (n)
    UNWIND labels(n) AS label
    RETURN label, count(*) AS count
    ORDER BY count DESC, label
    """
    return query(q)


### Q2. Relationship Types Overview
**Question**: Which relationship types exist and how many of each?  
**Approach**: `kg_relationship_types()` and `kg_relationship_counts()`  
**Why**: Identify ATT&CK-relevant edges (especially `ATTACK_REL` + `IN_TACTIC`).  
**Validate**: `ATTACK_REL` and `IN_TACTIC` should appear with non-trivial counts.


### Q3. ATT&CK STIX Type Distribution
**Question**: How many `:Attack` nodes per `stix_type`?  
**Approach**: `kg_attack_stix_distribution()`  
**Why**: See whether techniques, software, groups, mitigations are well represented.  
**Validate**: Techniques ~800, malware ~600+, groups ~180+, tactics ~14 (typical).


### Q4. Sample ATT&CK Nodes
**Question**: Show 10 example `:Attack` nodes with key fields.  
**Approach**: `kg_sample_nodes("Attack", limit=10)`  
**Why**: Confirm properties (`name`, `stix_type`, `id`) and get a feel for data quality.  
**Validate**: You should see diverse `stix_type` rows.


### Q5–Q9. Name Lists (Techniques / Groups / Software / Tactics / Mitigations)
**Question**: Print first 20 alphabetically for each category.  
**Approach**: Use `list_techniques()`, `list_groups()`, `list_software()`, `list_tactics()`, `list_mitigations()`  
**Why**: Build mental map; provide value pickers for demos.  
**Validate**: Check that software includes both `tool` and `malware`.


### Q10. Key ATT&CK Link Presence
**Question**: Do we have the critical edges (`uses`, `mitigates`, `subtechnique-of`, `IN_TACTIC`) and how many?  
**Approach**: `kg_reltype_property_counts()` + `kg_check_link_presence()`  
**Why**: Confirms the dataset can support multistep & algorithmic questions.  
**Validate**: `uses` should dominate; `mitigates` and `IN_TACTIC` present.


In [17]:

def kg_relationship_counts():
    q = """
    MATCH ()-[r]->()
    RETURN type(r) AS rel_type, count(*) AS count
    ORDER BY count DESC, rel_type
    """
    return query(q)

def kg_attack_stix_distribution():
    """
    If you have ATT&CK-style nodes, this shows stix_type distribution.
    Returns empty if :Attack label doesn't exist.
    """
    q = """
    MATCH (n:Attack)
    RETURN n.stix_type AS stix_type, count(*) AS count
    ORDER BY count DESC, stix_type
    """
    return query(q)

def kg_sample_nodes(label=None, limit=10):
    """
    Show sample nodes with their keys to see real properties.
    """
    if label:
        q = f"""
        MATCH (n:`{label}`)
        RETURN labels(n) AS labels, n.name AS name, n.id AS id, n.stix_type AS stix_type, keys(n) AS keys
        LIMIT $limit
        """
        return query(q, limit=int(limit))
    else:
        q = """
        MATCH (n)
        RETURN labels(n) AS labels, n.name AS name, n.id AS id, n.stix_type AS stix_type, keys(n) AS keys
        LIMIT $limit
        """
        return query(q, limit=int(limit))

def kg_reltype_property_counts():
    """
    If you use a generic relationship with a property (e.g., :ATTACK_REL {rel_type:'uses'}),
    this shows how many of each (rel_type) you have. Safe even if r.rel_type doesn't exist.
    """
    q = """
    MATCH ()-[r]->()
    WITH type(r) AS rel_type_name, r.rel_type AS rel_prop
    RETURN rel_type_name, rel_prop, count(*) AS count
    ORDER BY count DESC, rel_type_name, rel_prop
    """
    return query(q)

def kg_check_link_presence():
    """
    Counts of common ATT&CK edges, returned in a single row.
    """
    q = """
    OPTIONAL MATCH ()-[r0:ATTACK_REL]->()
    WITH count(r0) AS ATTACK_REL_total
    OPTIONAL MATCH ()-[r1:ATTACK_REL {rel_type:'uses'}]->()
    WITH ATTACK_REL_total, count(r1) AS ATTACK_REL_uses
    OPTIONAL MATCH ()-[r2:ATTACK_REL {rel_type:'mitigates'}]->()
    WITH ATTACK_REL_total, ATTACK_REL_uses, count(r2) AS ATTACK_REL_mitigates
    OPTIONAL MATCH ()-[r3:ATTACK_REL {rel_type:'subtechnique-of'}]->()
    WITH ATTACK_REL_total, ATTACK_REL_uses, ATTACK_REL_mitigates, count(r3) AS ATTACK_REL_subtech
    OPTIONAL MATCH ()-[r4:IN_TACTIC]->()
    RETURN ATTACK_REL_total, ATTACK_REL_uses, ATTACK_REL_mitigates, ATTACK_REL_subtech, count(r4) AS IN_TACTIC_total
    """
    return query(q)

# ============================================
# “Important” ATT&CK node types (names)
# ============================================

def list_techniques(term=None, limit=200):
    base = "MATCH (t:Attack {stix_type:'attack-pattern'})\n"
    params = {"limit": int(limit)}
    if term:
        base += "WHERE toLower(t.name) CONTAINS toLower($term)\n"
        params["term"] = term
    base += "RETURN t.name AS name ORDER BY name LIMIT $limit"
    return query(base, **params)

def list_groups(term=None, limit=200):
    base = "MATCH (g:Attack {stix_type:'intrusion-set'})\n"
    params = {"limit": int(limit)}
    if term:
        base += "WHERE toLower(g.name) CONTAINS toLower($term)\n"
        params["term"] = term
    base += "RETURN g.name AS name ORDER BY name LIMIT $limit"
    return query(base, **params)

def list_software(term=None, limit=200):
    base = "MATCH (s:Attack) WHERE s.stix_type IN ['tool','malware']\n"
    params = {"limit": int(limit)}
    if term:
        base += "AND toLower(s.name) CONTAINS toLower($term)\n"
        params["term"] = term
    base += "RETURN s.name AS name, s.stix_type AS kind ORDER BY name LIMIT $limit"
    return query(base, **params)

def list_tactics(term=None, limit=200):
    base = "MATCH (ta:Attack {stix_type:'x-mitre-tactic'})\n"
    params = {"limit": int(limit)}
    if term:
        base += "WHERE toLower(coalesce(ta.shortname, ta.name)) CONTAINS toLower($term)\n"
        params["term"] = term
    base += "RETURN coalesce(ta.shortname, ta.name) AS tactic ORDER BY tactic LIMIT $limit"
    return query(base, **params)

def list_mitigations(term=None, limit=200):
    base = "MATCH (m:Attack {stix_type:'course-of-action'})\n"
    params = {"limit": int(limit)}
    if term:
        base += "WHERE toLower(m.name) CONTAINS toLower($term)\n"
        params["term"] = term
    base += "RETURN m.name AS mitigation ORDER BY mitigation LIMIT $limit"
    return query(base, **params)

# ============================================
# One-call overview wrapper (handy to keep)
# ============================================

def kg_overview(limit=100):
    return {
        "labels": kg_labels(),
        "relationship_types": kg_relationship_types(),
        "node_counts_by_label": kg_node_counts_by_label(),
        "relationship_counts": kg_relationship_counts(),
        "attack_stix_distribution": kg_attack_stix_distribution(),
        "attack_samples": kg_sample_nodes("Attack", limit=10),
        "reltype_property_counts": kg_reltype_property_counts(),
        "key_link_presence": kg_check_link_presence(),
        "techniques_sample": list_techniques(limit=limit),
        "groups_sample": list_groups(limit=limit),
        "software_sample": list_software(limit=limit),
        "tactics_sample": list_tactics(limit=limit),
        "mitigations_sample": list_mitigations(limit=limit),
    }
# 1) Basic schema
labels_df         = kg_labels()
reltypes_df       = kg_relationship_types()
nodes_by_label_df = kg_node_counts_by_label()
rels_by_type_df   = kg_relationship_counts()

labels_df, reltypes_df, nodes_by_label_df.head(20), rels_by_type_df.head(20)


(        label
 0      Attack
 1      Entity
 2    FDEntity
 3  FDRelation
 4      FDTime
 5    Relation
 6        Time,
           relationshipType
 0                  ACCEPTS
 1              ACCESSEDVIA
 2             ACCOUNTEDFOR
 3     ACCURATELY_PREDICTED
 4                  ACHIEVE
 ...                    ...
 1601                wastes
 1602               weakens
 1603                  will
 1604                  work
 1605                 wrote
 
 [1606 rows x 1 columns],
         label  count
 0    FDEntity  13645
 1      Entity   5873
 2      Attack   2242
 3      FDTime    261
 4  FDRelation     15
 5    Relation     15,
         rel_type   count
 0        FD_FACT  223983
 1     ATTACK_REL   20411
 2      IN_TACTIC    1076
 3      RELATEDTO     575
 4     related_to     388
 5      relatedto     320
 6     hascontext     300
 7             IS     217
 8             is     204
 9           HAVE     195
 10    HASCONTEXT     193
 11          have     190
 12  INFLUENCEDBY     

In [19]:
# 2) ATT&CK-specific views (safe even if :Attack missing; just returns empty)
stix_dist_df      = kg_attack_stix_distribution()
attack_samples_df = kg_sample_nodes("Attack", limit=10)
relprop_counts_df = kg_reltype_property_counts()
link_presence_df  = kg_check_link_presence()

stix_dist_df, attack_samples_df, relprop_counts_df.head(20), link_presence_df


(                 stix_type  count
 0           attack-pattern    823
 1                  malware    667
 2         course-of-action    268
 3            intrusion-set    181
 4   x-mitre-data-component    109
 5                     tool     91
 6                 campaign     47
 7      x-mitre-data-source     38
 8           x-mitre-tactic     14
 9                 identity      1
 10      marking-definition      1
 11      x-mitre-collection      1
 12          x-mitre-matrix      1,
      labels                                           name  \
 0  [Attack]                              Enterprise ATT&CK   
 1  [Attack]                              Enterprise ATT&CK   
 2  [Attack]                 Password Filter DLL Mitigation   
 3  [Attack]                Space after Filename Mitigation   
 4  [Attack]                         HISTCONTROL Mitigation   
 5  [Attack]                Credentials in Files Mitigation   
 6  [Attack]  Exploitation for Credential Access Mitigation   
 7  [

# Level 2 — Multistep (analytics & joins)

### Q1. Software Overlap for Technique Name (e.g., “malicious file”)
**Question**: Which software are used by multiple groups that also use the technique(s) whose name contains a term?  
**Approach**:
1) Find techniques by name: `list_attack_patterns(term)`
2) Groups using them: `list_groups_using_attack_pattern(term)`
3) Software used by those groups: `list_software_used_by_those_groups(term)`
4) Keep software shared by >1 group  


In [27]:
# =========================
# Q1 — Adversary Group & Software Overlap (T1204.002)
# =========================
def Q1_overlap_for_malicious_file(tech_name="Malicious File", min_groups=2, match="exact"):
    pred = {
        "exact":   "toLower(t.name) = toLower($tech_name)",
        "contains":"toLower(t.name) CONTAINS toLower($tech_name)",
        "regex":   "t.name =~ $tech_name"
    }[match]
    q = f"""
    MATCH (t:Attack {{stix_type:'attack-pattern'}})
    WHERE {pred}
    MATCH (g:Attack {{stix_type:'intrusion-set'}})-[:ATTACK_REL {{rel_type:'uses'}}]->(t)
    MATCH (g)-[:ATTACK_REL {{rel_type:'uses'}}]->(s:Attack)
    WHERE s.stix_type IN ['tool','malware']
    WITH s, collect(DISTINCT g.name) AS groups, count(DISTINCT g) AS group_count
    WHERE group_count >= $min_groups
    RETURN s.name AS software, s.stix_type AS kind, group_count, groups
    ORDER BY group_count DESC, software
    """
    return query(q, tech_name=tech_name, min_groups=int(min_groups))


Q1_overlap_for_malicious_file("Malicious File", min_groups=2, match="exact")


,software,kind,group_count,groups
0,Mimikatz,tool,26,"[Indrik Spider, Wizard Spider, FIN7, Dragonfly..."
1,Cobalt Strike,malware,18,"[Indrik Spider, Wizard Spider, FIN7, TA505, Ea..."
2,Net,tool,18,"[Wizard Spider, Dragonfly, OilRig, TA505, admi..."
3,PsExec,tool,16,"[Indrik Spider, Wizard Spider, Dragonfly, OilR..."
4,Empire,tool,11,"[Indrik Spider, Wizard Spider, WIRTE, Sandworm..."
...,...,...,...,...
64,at,tool,2,"[BRONZE BUTLER, Leviathan]"
65,cmd,tool,2,"[BRONZE BUTLER, menuPass]"
66,ngrok,tool,2,"[OilRig, LazyScripter]"
67,reGeorg,malware,2,"[APT28, APT29]"


### Q2. Top-3 Techniques for a Group by Software Implementations + Mitigations
**Question**: For a given group (e.g., APT28), which 3 techniques are implemented by the largest number of distinct software, and what mitigations exist for those techniques?  
**Approach**:
- Group → `uses` → Technique  
- Count Software → `uses` → same Technique  
- ORDER BY count DESC, LIMIT 3  
- Technique ← `mitigates` — Mitigation  
**Helper**: `Q2_top_3_tech_mitigations_for_group("APT28")` (fixed version).



In [35]:
# =========================
# Q2 — Mitigations for Group's Top-3 Techniques by Software Implementations
# =========================

def Q2_top3_tech_mitigations_for_group(group_name="APT28"):
    """
    For `group_name`, find techniques they use, rank by the number of software
    (tool|malware) that implement each technique, take top-3, then list mitigations.
    Mitigations are collected for the technique itself AND its parent (if it's a sub-tech).
    """
    q = """
    // -- anchor group
    WITH toLower($group) AS gname
    MATCH (grp:Attack {stix_type:'intrusion-set'})
    WHERE toLower(grp.name) CONTAINS gname

    // -- techniques the group uses
    MATCH (grp)-[:ATTACK_REL {rel_type:'uses'}]->(tech:Attack {stix_type:'attack-pattern'})

    // -- count software implementing each technique (tool|malware)
    OPTIONAL MATCH (s:Attack)-[:ATTACK_REL {rel_type:'uses'}]->(tech)
    WHERE s.stix_type IN ['tool','malware']
    WITH tech, count(DISTINCT s) AS software_count
    ORDER BY software_count DESC, tech.name
    LIMIT 3

    // -- mitigations directly on the technique
    OPTIONAL MATCH (co:Attack {stix_type:'course-of-action'})-[:ATTACK_REL {rel_type:'mitigates'}]->(tech)
    WITH tech, software_count, collect(DISTINCT co.name) AS mit_direct

    // -- if tech is a sub-technique, find its parent and add parent mitigations
    OPTIONAL MATCH (tech)-[:ATTACK_REL {rel_type:'subtechnique-of'}]->(parent:Attack {stix_type:'attack-pattern'})
    OPTIONAL MATCH (co2:Attack {stix_type:'course-of-action'})-[:ATTACK_REL {rel_type:'mitigates'}]->(parent)
    WITH tech, software_count, mit_direct, collect(DISTINCT co2.name) AS mit_parent

    // -- merge & return
    WITH tech, software_count,
         [m IN mit_direct WHERE m IS NOT NULL] + [m IN mit_parent WHERE m IS NOT NULL] AS mit_all
    RETURN tech.name AS technique,
           software_count,
           CASE WHEN size(mit_all)=0 THEN [] ELSE apoc.coll.toSet(mit_all) END AS mitigations
    ORDER BY software_count DESC, technique
    """
    return query(q, group=group_name)


Q2_top3_tech_mitigations_for_group("APT28")

,technique,software_count,mitigations
0,Ingress Tool Transfer,374,[Network Intrusion Prevention]
1,Web Protocols,317,"[Network Intrusion Prevention, Filter Network ..."
2,File and Directory Discovery,288,[]


### Q3. Most-used Sub-techniques Under `execution` and Who Uses Them
**Question**: Which sub-techniques under the `execution` tactic are most used, and by which groups?  
**Approach**:
- Technique −[:IN_TACTIC]→ Tactic(`shortname='execution'`)
- Groups −[:uses]→ Technique
- Aggregate by sub-technique name; collect top groups  

In [37]:
# =========================
# Q3 — Sub-technique Vulnerability Patterns under Execution
# =========================
def Q3_execution_subtechniques_usage(limit=50):
    q = """
    MATCH (tac:Attack {stix_type:'x-mitre-tactic', shortname:'execution'})
    MATCH (sub:Attack {stix_type:'attack-pattern'})-[:IN_TACTIC]->(tac)
    // ensure sub-technique (has a parent technique)
    WHERE EXISTS {
      (sub)-[:ATTACK_REL {rel_type:'subtechnique-of'}]->(:Attack {stix_type:'attack-pattern'})
    }
    OPTIONAL MATCH (g:Attack {stix_type:'intrusion-set'})-[:ATTACK_REL {rel_type:'uses'}]->(sub)
    WITH sub, collect(DISTINCT g.name) AS groups, count(DISTINCT g) AS groups_using
    RETURN sub.name AS subtechnique, groups_using, groups
    ORDER BY groups_using DESC, subtechnique
    LIMIT $limit
    """
    return query(q, limit=int(limit))

Q3_execution_subtechniques_usage()


,subtechnique,groups_using,groups
0,Malicious File,83,"[APT19, HEXANE, TA551, Silence, BRONZE BUTLER,..."
1,PowerShell,79,"[Ember Bear, Mustang Panda, Leviathan, BRONZE ..."
2,Windows Command Shell,68,"[TA505, Indrik Spider, Aquatic Panda, Cinnamon..."
3,Scheduled Task,52,"[Naikon, Higaisa, Daggerfly, Cobalt Group, APT..."
4,Malicious Link,46,"[Sidewinder, Mofang, Gamaredon Group, Leviatha..."
5,Visual Basic,44,"[Leviathan, RedCurl, BRONZE BUTLER, SideCopy, ..."
6,JavaScript,23,"[Leafminer, Higaisa, Evilnum, Turla, TA505, Si..."
7,Python,15,"[ZIRCONIUM, Machete, RedCurl, Kimsuky, BRONZE ..."
8,Service Execution,14,"[BlackByte, FIN6, APT39, APT41, Moonstone Slee..."
9,Dynamic Data Exchange,11,"[Gallmaker, MuddyWater, Leviathan, Patchwork, ..."


### Q4. Shared Tactics Between Dissimilar Software (e.g., “rar” vs “PsExec”)
**Question**: Which tactics are accomplished by techniques implemented by both software A and B?  
**Approach**:
- S1 −[:uses]→ Technique ←[:IN_TACTIC]→ Tactic
- S2 −[:uses]→ Technique ←[:IN_TACTIC]→ Tactic
- Intersect tactics; show example techniques per side  
**Helpers**: 
- `software_tactics("rar")`, `software_tactics("psexec")`  
- `Q4_shared_tactics_between_software("rar","psexec")`  



In [41]:
def top_software_by_techniques(top=25):
    """
    Software (tool|malware) ranked by how many techniques they implement.
    Handy for choosing a second software that will definitely intersect.
    """
    q = """
    MATCH (s:Attack)
    WHERE s.stix_type IN ['tool','malware']
    MATCH (s)-[:ATTACK_REL {rel_type:'uses'}]->(t:Attack {stix_type:'attack-pattern'})
    RETURN s.name AS software, s.stix_type AS kind, count(DISTINCT t) AS techniques_count
    ORDER BY techniques_count DESC, software
    LIMIT $top
    """
    return query(q, top=int(top))


def software_tactics(software_term, limit=200):
    """
    Tactics covered by techniques of software whose name CONTAINS software_term.
    """
    q = """
    MATCH (s:Attack)
    WHERE s.stix_type IN ['tool','malware'] AND toLower(s.name) CONTAINS toLower($term)
    MATCH (s)-[:ATTACK_REL {rel_type:'uses'}]->(t:Attack {stix_type:'attack-pattern'})
    MATCH (t)-[:IN_TACTIC]->(ta:Attack {stix_type:'x-mitre-tactic'})
    RETURN s.name AS software, s.stix_type AS kind,
           collect(DISTINCT ta.shortname) AS tactics,
           count(DISTINCT ta) AS n_tactics
    ORDER BY n_tactics DESC, software
    LIMIT $limit
    """
    return query(q, term=software_term, limit=int(limit))


def Q4_shared_tactics_between_software(s1_term, s2_term, examples_per_side=8):
    """
    Intersect tactics achieved by techniques implemented by BOTH s1_term and s2_term.
    Uses CONTAINS match so you can pass 'rar', 'mimi', etc.
    """
    q = """
    WITH toLower($s1) AS a, toLower($s2) AS b

    MATCH (s1:Attack) WHERE s1.stix_type IN ['tool','malware'] AND toLower(s1.name) CONTAINS a
    MATCH (s2:Attack) WHERE s2.stix_type IN ['tool','malware'] AND toLower(s2.name) CONTAINS b

    MATCH (s1)-[:ATTACK_REL {rel_type:'uses'}]->(t1:Attack {stix_type:'attack-pattern'})-[:IN_TACTIC]->(ta:Attack {stix_type:'x-mitre-tactic'})
    MATCH (s2)-[:ATTACK_REL {rel_type:'uses'}]->(t2:Attack {stix_type:'attack-pattern'})-[:IN_TACTIC]->(ta)

    WITH s1.name AS s1_name, s2.name AS s2_name, ta.shortname AS tactic,
         collect(DISTINCT t1.name)[0..$k] AS s1_tech_examples,
         collect(DISTINCT t2.name)[0..$k] AS s2_tech_examples
    RETURN s1_name, s2_name, tactic, s1_tech_examples, s2_tech_examples
    ORDER BY tactic
    """
    return query(q, s1=s1_term, s2=s2_term, k=int(examples_per_side))


def Q4_shared_tactics_with_diagnostics(s1_term, s2_term):
    """
    Convenience wrapper: returns (s1_tactics, s2_tactics, shared_tactics).
    If shared is empty, you still see each side's tactic sets.
    """
    s1 = software_tactics(s1_term)
    s2 = software_tactics(s2_term)
    shared = Q4_shared_tactics_between_software(s1_term, s2_term)
    return {"s1_tactics": s1, "s2_tactics": s2, "shared": shared}


# See good candidates:
top_software_by_techniques(20)

# Sanity-check each side:
software_tactics("rar")        # you said this returns data
software_tactics("psexec")     # or try "mimi", "psexec", "powershell"


Q4_shared_tactics_between_software("rar", "psexec")

,s1_name,s2_name,tactic,s1_tech_examples,s2_tech_examples
0,RARSTONE,PsExec,privilege-escalation,[Dynamic-link Library Injection],[Windows Service]


### Q5. Unmitigated Software Risks
**Question**: Which software implements techniques that have **no** documented mitigations and are used by at least one group?  
**Approach**:
- Technique with `NOT (co:course-of-action)-[:mitigates]->(tech)`
- Exists `(grp:intrusion-set)-[:uses]->(tech)`
- Exists `(soft:tool|malware)-[:uses]->(tech)`  
**Deliverable**: `software`, `technique`, `example_group`  

In [43]:
# =========================
# Q5 — Unmitigated Software Risks
# =========================
def Q5_unmitigated_software_risks(limit=50):
    q = """
    // techniques with NO mitigations
    MATCH (tech:Attack {stix_type:'attack-pattern'})
    WHERE NOT EXISTS {
      (:Attack {stix_type:'course-of-action'})-[:ATTACK_REL {rel_type:'mitigates'}]->(tech)
    }

    // techniques that at least one group uses
    MATCH (g:Attack {stix_type:'intrusion-set'})-[:ATTACK_REL {rel_type:'uses'}]->(tech)

    // software that implement those techniques
    MATCH (s:Attack)-[:ATTACK_REL {rel_type:'uses'}]->(tech)
    WHERE s.stix_type IN ['tool','malware']

    RETURN s.name AS software,
           s.stix_type AS kind,
           count(DISTINCT tech) AS unmitigated_tech_count,
           collect(DISTINCT tech.name)[0..20] AS unmitigated_techniques,
           collect(DISTINCT g.name)[0..20] AS example_groups
    ORDER BY unmitigated_tech_count DESC, software
    LIMIT $limit
    """
    return query(q, limit=int(limit))

Q5_unmitigated_software_risks()


,software,kind,unmitigated_tech_count,unmitigated_techniques,example_groups
0,InvisiMole,malware,27,"[Local Data Staging, Query Registry, System Se...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
1,QakBot,malware,24,"[Remote System Discovery, Local Data Staging, ...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
2,Machete,malware,21,"[Local Data Staging, Hidden Files and Director...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
3,SILENTTRINITY,tool,20,"[Remote System Discovery, Query Registry, Syst...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
4,FunnyDream,malware,19,"[Remote System Discovery, Local Data Staging, ...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
5,Bazar,malware,18,"[Remote System Discovery, Query Registry, Syst...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
6,DarkGate,malware,18,"[Hidden Files and Directories, Registry Run Ke...","[LuminousMoth, Wizard Spider, FIN7, Velvet Ant..."
7,Attor,malware,17,"[Local Data Staging, Query Registry, Hidden Fi...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
8,Cobalt Strike,malware,17,"[Remote System Discovery, Query Registry, Syst...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."
9,Crimson,malware,17,"[Query Registry, System Owner/User Discovery, ...","[Indrik Spider, LuminousMoth, Wizard Spider, F..."


### Q6. Group Technique → Tactic Coverage
**Question**: For techniques used by a given group, list the unique tactics they belong to.  
**Approach**:
- Group −[:uses]→ Technique −[:IN_TACTIC]→ Tactic
- Aggregate by tactic: count distinct techniques; show examples  
**Helper**: `group_tactic_coverage("APT29", examples_per_tactic=5)`  


In [62]:
def group_tactic_coverage(group_name="APT29", examples_per_tactic=5):
    """
    For the given group, list tactics covered by techniques they use,
    with a count of distinct techniques per tactic and a few examples.
    """
    q = """
    WITH toLower($g) AS gname, toInteger($k) AS K
    MATCH (g:Attack {stix_type:'intrusion-set'})
    WHERE toLower(g.name) CONTAINS gname

    MATCH (g)-[:ATTACK_REL {rel_type:'uses'}]->(t:Attack {stix_type:'attack-pattern'})
    MATCH (t)-[:IN_TACTIC]->(ta:Attack {stix_type:'x-mitre-tactic'})

    WITH ta.shortname AS tactic, collect(DISTINCT t.name) AS techs, K
    RETURN tactic,
           size(techs) AS techniques_count,
           techs[0..K] AS example_techniques
    ORDER BY techniques_count DESC, tactic
    """
    return query(q, g=group_name, k=int(examples_per_tactic))


group_tactic_coverage("APT29", examples_per_tactic=5)



,tactic,techniques_count,example_techniques
0,defense-evasion,15,"[Mshta, Timestomp, Hybrid Identity, Local Acco..."
1,persistence,15,"[Registry Run Keys / Startup Folder, Scheduled..."
2,privilege-escalation,13,"[Boot or Logon Initialization Scripts, Valid A..."
3,execution,9,"[PowerShell, Cloud Administration Command, Mal..."
4,initial-access,9,"[Local Accounts, Cloud Accounts, Trusted Relat..."
5,credential-access,8,"[Security Account Manager, Steal or Forge Auth..."
6,command-and-control,7,"[Dynamic Resolution, Ingress Tool Transfer, En..."
7,resource-development,6,"[Tool, Web Services, Cloud Accounts, Email Acc..."
8,collection,2,"[Data from Local System, Remote Email Collection]"
9,discovery,2,"[Cloud Account, Internet Connection Discovery]"


# Level 3 — Graph-Algorithm flavored

> If **GDS** is available, use native algorithms (PageRank, Louvain, Betweenness).  
> If not, use the included **fallbacks** (degree proxy, connected components, etc.).

### GA1. Centrality of Techniques (Pivot Points)
**Question**: Which techniques are most “influential” across the KG?  
**GDS Approach**: Project `intrusion-set`, `tool`, `malware`, `attack-pattern` with `uses` edges; run PageRank; return top techniques.  
**Fallback**: Degree proxy = `#groups + #software` linked to the technique.  


In [50]:
# GA1 — Centrality (PageRank fallback to degree)
def GA1_top_techniques_pagerank(top_n=25):
        q = """
        MATCH (t:Attack {stix_type:'attack-pattern'})
        OPTIONAL MATCH (t)<-[:ATTACK_REL {rel_type:'uses'}]-(:Attack)
        WITH t, count(*) AS indeg
        OPTIONAL MATCH (t)-[:ATTACK_REL {rel_type:'uses'}]->(:Attack)
        WITH t, indeg, count(*) AS outdeg
        RETURN t.name AS technique, (indeg + outdeg) AS degree_centrality
        ORDER BY degree_centrality DESC
        LIMIT $top
        """
        return query(q, top=int(top_n))


GA1_top_techniques_pagerank()


,technique,degree_centrality
0,Ingress Tool Transfer,483
1,System Information Discovery,422
2,Web Protocols,391
3,Windows Command Shell,363
4,File and Directory Discovery,344
5,Deobfuscate/Decode Files or Information,304
6,Process Discovery,291
7,File Deletion,287
8,System Network Configuration Discovery,267
9,Registry Run Keys / Startup Folder,245


### GA2. Communities (“Attack Kits”) — Groups + Software + Techniques
**Question**: Are there clusters of groups, software, and techniques that move together?  
**GDS Approach**: Louvain on the projected `uses` graph.  
**Fallback**: Fast connected-component style using only `uses` edges (helpers provided):
- `GA3_communities_uses_fast(limit=?, max_hops=?, seed_limit=?)`  
**Deliverable**: Top communities with counts per type and example members.  


In [56]:
def GA2_communities_uses_fast(limit=50, max_hops=4, seed_limit=120):
    """
    Fast, APOC-free fallback for 'attack kit' communities.
    - Seeds from intrusion sets only (keeps workload small).
    - Traverses ONLY ATTACK_REL edges whose rel_type='uses' (enforced per-path).
    - Traversal depth bounded by max_hops.
    - Dedupes components by stable key (min elementId across component).
    """
    q = f"""
    // 1) Seed from groups
    MATCH (seed:Attack {{stix_type:'intrusion-set'}})
    WITH seed
    ORDER BY seed.name
    LIMIT $seed_limit

    // 2) 'uses'-only neighborhood (bounded)
    CALL {{
      WITH seed
      MATCH p = (seed)-[:ATTACK_REL*1..{int(max_hops)}]-(m:Attack)
      WHERE m.stix_type IN ['intrusion-set','tool','malware','attack-pattern']
        AND all(r IN relationships(p) WHERE r.rel_type = 'uses')
      WITH seed, collect(DISTINCT m) AS ms
      RETURN ms + [seed] AS nodes            // include seed itself (no agg conflict)
    }}

    // 3) Compute a stable component key for these nodes
    UNWIND nodes AS n
    WITH nodes, min(elementId(n)) AS component_key
    WITH component_key, nodes

    // 4) Merge duplicates: if multiple seeds land in same component,
    //    union all members under the same component_key
    UNWIND nodes AS n
    WITH component_key, collect(DISTINCT n) AS comp_nodes

    // 5) Split, count, and return
    WITH component_key,
         [x IN comp_nodes WHERE x.stix_type='intrusion-set'            | x.name] AS groups,
         [x IN comp_nodes WHERE x.stix_type IN ['tool','malware']      | x.name] AS software,
         [x IN comp_nodes WHERE x.stix_type='attack-pattern'           | x.name] AS techniques
    WITH component_key,
         size(groups)     AS n_groups,
         size(software)   AS n_software,
         size(techniques) AS n_techniques,
         groups, software, techniques
    RETURN component_key,
           n_groups,
           n_software,
           n_techniques,
           (n_groups + n_software + n_techniques) AS size_total,
           groups[0..10]     AS example_groups,
           software[0..10]   AS example_software,
           techniques[0..10] AS example_techniques
    ORDER BY size_total DESC, n_groups DESC, n_software DESC, n_techniques DESC
    LIMIT $limit
    """
    return query(q, seed_limit=int(seed_limit), limit=int(limit))


GA2_communities_uses_fast(limit=25, max_hops=4, seed_limit=120)


,component_key,n_groups,n_software,n_techniques,size_total,example_groups,example_software,example_techniques
0,4:0d0c2757-2ba2-470f-9751-377bca5b6e4c:6143,166,755,574,1495,"[Winter Vivern, Tropic Trooper, menuPass, Agri...","[Micropsia, Metamorfo, Ramsay, AppleSeed, MESS...","[Audio Capture, File and Directory Discovery, ..."


### GA3. Link Prediction: Likely Next Techniques for a Group
**Question**: Given a group’s software set, which techniques are they most likely to use next?  
**Approach** (Adamic-Adar-like):
- From group’s software neighbors to candidate techniques not yet used
- Weight each software by inverse log of its “degree” to techniques
- Add a small social signal: other groups that use both the software and the candidate  
**Helper**: `GA4_link_prediction_for_group("APT1", top_n=20)` 


In [58]:
# GA3 — Link prediction: which techniques a group is likely to use next?
def GA3_link_prediction_for_group(group, top_n=20):
    q = """
    WITH toLower($group) AS gname

    // 1) Resolve the group and its software set
    MATCH (g:Attack {stix_type:'intrusion-set'})
    WHERE toLower(g.name) CONTAINS gname
    MATCH (g)-[:ATTACK_REL {rel_type:'uses'}]->(s:Attack)
    WHERE s.stix_type IN ['tool','malware']
    WITH g, toLower(g.name) AS gname_lower, collect(DISTINCT s) AS g_soft

    // 2) Candidate techniques via those software that g does not (yet) use
    UNWIND g_soft AS s
    MATCH (s)-[:ATTACK_REL {rel_type:'uses'}]->(cand:Attack {stix_type:'attack-pattern'})
    WHERE NOT (g)-[:ATTACK_REL {rel_type:'uses'}]->(cand)

    // 3) Software degree for Adamic–Adar style weighting (over technique neighbors)
    OPTIONAL MATCH (s)-[:ATTACK_REL {rel_type:'uses'}]->(st:Attack {stix_type:'attack-pattern'})
    WITH g, gname_lower, cand, s, count(st) AS s_deg
    // Avoid log(0): use log10(s_deg + 2.0), so minimum denominator > 0
    WITH g, gname_lower, cand, s, 1.0 / log10(toFloat(s_deg) + 2.0) AS w

    // 4) Aggregate per candidate: AA weight and keep the software set that votes for it
    WITH g, gname_lower, cand,
         sum(w) AS aa_software_weight,
         collect(DISTINCT s) AS s_set

    // 5) Social support: other groups that use any s in s_set AND the same candidate technique
    UNWIND s_set AS s2
    OPTIONAL MATCH (other:Attack {stix_type:'intrusion-set'})-[:ATTACK_REL {rel_type:'uses'}]->(s2)
    OPTIONAL MATCH (other)-[:ATTACK_REL {rel_type:'uses'}]->(cand)
    WITH gname_lower, cand, aa_software_weight, collect(DISTINCT toLower(other.name)) AS other_names
    WITH cand,
         aa_software_weight,
         size([n IN other_names WHERE n IS NOT NULL AND n <> gname_lower]) AS other_groups_support

    // 6) Final score and output
    RETURN cand.name AS technique,
           round(aa_software_weight, 4) AS aa_weight,
           other_groups_support,
           round(aa_software_weight + 0.1 * other_groups_support, 4) AS score
    ORDER BY score DESC, technique
    LIMIT $top
    """
    return query(q, group=group, top=int(top_n))


GA3_link_prediction_for_group("APT1")

,technique,aa_weight,other_groups_support,score
0,Security Account Manager,4.5389,49,9.4389
1,Service Execution,4.0758,51,9.1758
2,Windows Service,2.9226,47,7.6226
3,LSA Secrets,2.4430,48,7.2430
4,Domain Account,1.9799,51,7.0799
5,SMB/Windows Admin Shares,1.9799,51,7.0799
6,Account Manipulation,0.7820,48,5.5820
7,Credentials from Password Stores,0.7820,48,5.5820
8,Credentials from Web Browsers,0.7820,48,5.5820
9,DCSync,0.7820,48,5.5820
